In [ ]:
#import libraries

from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
import json
import os
import requests
from langchain.tools import tool
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from crewai_tools import tool



In [ ]:
# definition of search tool for agents

search_tool = DuckDuckGoSearchRun()


In [ ]:


# enter api key here

os.environ["OPENAI_API_KEY"] = api_key

In [ ]:

# creation of vectordatabase for patent database  

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loaders = [
    PyPDFLoader("trefferliste_BASF_no_duplicates.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

# Split docs into chunks
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(api_key = api_key)

from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(splits, embedding)

from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain


#queries using ConversationalRetrieval Chain

chain = ConversationalRetrievalChain.from_llm(llm = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo', api_key=api_key), chain_type = "stuff", retriever=vectorstore.as_retriever(), verbose = True)



In [ ]:
#creation of tool for searching in patent database

@tool("patent_database_search")
def patent_database_search(question: str) -> str:
    """This tool can search in the patent database by exclusively screening the context provided."""
    return chain.invoke({"question": question, "chat_history": []})

In [ ]:
from crewai import Agent

# Define agents for first crew that needs to engage in product ideation

topic_identification_specialist = Agent(
    role='Topic Identification Specialist',
    goal='Summarize the three most common topics in the provided patent database',
    backstory="""You are a great expert in summarizing topics from the provided patent database.""",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
    tools=[patent_database_search]
)

product_development_specialist = Agent(
    role='Product Development Specialist',
    goal='Use the identified topics from the topic_identification_specialist and the information of the patent_database_search tool to explore a novel product that can leverage the topics and patents.',
    backstory="""You are a very experienced product developer that can come up with creative suggestions for new products based on recombining available patents.""",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
    tools=[patent_database_search]
)



In [ ]:
# Create tasks for the agents
get_topics = Task(
  description="""Summarize the three most common topics in the provided patent database.""",
  expected_output="""As output, you provide a list of three topics that are present in multiple patents in the database and that have potential for developing a novel product.""",
  agent=topic_identification_specialist
)

get_product_idea = Task(
  description=f"""Use the patent_database_search tool and the topics identified by the topic_identification_specialist to ideate on novel product. You should ask the patent_database_search tool questions such as: Question 1: What are potential painpoints that can be solved with the patents in the database?; Question 2: What could be an interesting proudct that can be developed by recombining the patents in the database? The product should be very specific and leverage specific aspects of the patents. You are allowed to engage in multiple iterations to fine-tune and optimize the product idea.""",
  expected_output='As output, provide the following two elements: (i) a clear and compelling description of the product and (ii) Provide a list of the patents that are used in the product idea.',
  agent=product_development_specialist
)




In [ ]:
from crewai import Crew, Process

# Instantiate the first crew with a sequential process
crew = Crew(
    agents=[topic_identification_specialist, product_development_specialist],
    tasks=[get_topics, get_product_idea],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
)

In [ ]:
# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

In [ ]:
print(results['final_output'])

In [ ]:
initial_idea = results['final_output']

In [ ]:
#intiate agents for second crew that needs to optimize the product idea

technical_expert = Agent(
    role='Technical Expert',
    goal='You can optimize a product idea by analyzing its technological feasibility and innovativeness.',
    backstory="""You are a technical expert that can optimize a product idea based on the technical feasibility of the product idea.""",
    verbose=True,
    allow_delegation=False,
    max_iter=3,
    memory=True,
    tools=[patent_database_search, search_tool]
)

competition_expert = Agent(
    role='Competition Expert',
    goal='You can search on the internet for competing products and optimize the product to make it as unique as possible leveraging the available patents.',
    backstory="""You are an expert in identifying competing products by search on the internet.""",
    verbose=True,
    allow_delegation=False,
    max_iter=3,
    memory=True,
    tools=[patent_database_search, search_tool]
)

finalization_expert = Agent(
    role='Finalization Expert',
    goal='You can finalize the product idea by providing a final product description based on the optimizations of the technical_expert and competition_expert.',
    backstory="""You are an expert in finalizing product ideas based on the technical feasibility and competition analysis.""",
    verbose=True,
    allow_delegation=False,
    max_iter=3,
    memory=True,
)


In [ ]:
# Create tasks for the agents

technical_optimization = Task(
    description=f"""Engage in a technical optimization of the following product idea: {initial_idea}.""",
    expected_output='As output, provide an optimized description of the product idea.',
    agent=technical_expert,
)

competition_optimization = Task(
    description=f"""Engage in a competition-driven optimization where you make the following product idea as distinctive as possible: {initial_idea}.""",
    expected_output='As output, provide an optimized description of the product idea.',
    agent=competition_expert,
)

finalization = Task(
    description=f"""Finalize the product idea by providing a final product description based on the optimizations of the technical_expert and competition_expert.""",
    expected_output='As output, provide a final description of the product idea.',
    agent=finalization_expert,
)


In [ ]:
# Instantiate the second crew with a hierarchical process

second_crew = Crew(
    agents=[technical_expert, competition_expert, finalization_expert],
    tasks=[technical_optimization, competition_optimization, finalization],
    verbose=2,
    manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
)

In [ ]:
# Kick off the crew's work
second_results = second_crew.kickoff()

# Print the results
print("Crew Work Results:")
print(second_results)

In [ ]:
print(second_results['final_output'])

In [ ]:
print(f"""
    Output technical optimization: {technical_optimization.output.raw_output}
    Output competition optimization: {competition_optimization.output.raw_output}
    Output finalization: {finalization.output.raw_output}
""")